<a href="https://colab.research.google.com/github/muniappabalaji/TimeSeries/blob/main/LLM/RAG/RAG_Pine_Cone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip install -q pinecone pinecone-client sentence-transformers langchain langchain_community langchain_experimental langchain_openai langchain-pinecone pdfplumber

In [16]:
# !git clone https://github.com/deepanrajm/GL.git
!git clone https://github.com/muniappabalaji/TimeSeries.git

fatal: destination path 'TimeSeries' already exists and is not an empty directory.


In [17]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Make sure the PDF path is correct
pdf_path = r"/content/GL/LLM/RAG/Basic_Home_Remedies.pdf"
loader = PDFPlumberLoader(pdf_path)
docs = loader.load()

print(f"Number of pages in the PDF: {len(docs)}")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", ".", " ", ""]
)
documents = text_splitter.split_documents(docs)
print(f"Number of document chunks created: {len(documents)}")

Number of pages in the PDF: 3
Number of document chunks created: 9


In [18]:
import os
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
from google.colab import userdata
import time
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import Pinecone as PineconeVectorStore

# Securely get the API key from Colab secrets
try:
    PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
except userdata.SecretNotFoundError:
    print("ERROR: PINECONE_API_KEY not found in Colab secrets.")
    print("Please follow Step 1 in the instructions to add your key.")
    # Exit the script if the key is not found
    exit()

index_name = "Medical-rag"

# Initialize the Pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)
print(f"--- Pinecone initialized for index: '{index_name}' ---")

--- Pinecone initialized for index: 'Medical-rag' ---


In [19]:
# Instantiate the embedding model (same as your code)
embedder = HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2')

# Get the dimension of the embeddings
embedding_dim = len(embedder.embed_query("test query"))

# Create the index if it doesn't already exist
index_name = "medical-rag" # Changed index name to lowercase and hyphen

if index_name not in pc.list_indexes().names():
    print(f"Creating new Pinecone index: {index_name}")
    pc.create_index(
        name=index_name,
        dimension=embedding_dim,
        metric="cosine",
        spec=ServerlessSpec(cloud='aws', region='us-east-1')
    )
    print("Index created successfully.")
else:
    print(f"Index '{index_name}' already exists.")

Index 'medical-rag' already exists.


In [23]:
import os

print("\nCreating Pinecone vector store and populating it with documents...")

# Set the Pinecone API key as an environment variable
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

# The from_documents method will create embeddings and upsert them into the Pinecone index
vector_store = PineconeVectorStore.from_documents(
    documents,
    embedder,
    index_name=index_name
)

retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 1})
print("Vector store and retriever are ready.")


Creating Pinecone vector store and populating it with documents...
Vector store and retriever are ready.


In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(openai_api_base = "https://openrouter.ai/api/v1", openai_api_key = "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", model = "deepseek/deepseek-chat-v3.1:free",temperature=0.7) #max_tokens=100

# Your custom prompt template
prompt = """1. You are a doctor.
2. Use the following pieces of context to answer the question at the end.
3. Answer only by using the context and articulate it better, use bullet points and emoji if required.
4. Keep the answer crisp and limited to 3-4 sentences.
5.Don't say based on the context provided
Context: {context}
Question: {question}
Answer to the question:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt)

# Your chain setup
llm_chain = LLMChain(llm=llm, prompt=QA_CHAIN_PROMPT, verbose=False)
document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)
combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
)
qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    verbose=True,
    retriever=retriever,
    return_source_documents=True,
)

In [28]:
print("\n--- Sending query to the RAG chain ---")
result = qa("remedy for cough?")
print("\n--- Final Answer ---")
print(result["result"])
print("\n--- Retrieved Documents ---")
for doc in result["source_documents"]:
    print(f"Content: {doc.page_content}\nSource: {doc.metadata.get('source', 'N/A')}\n---")


--- Sending query to the RAG chain ---


> Entering new RetrievalQA chain...

> Finished chain.

--- Final Answer ---
Based on the context provided, here are remedies for a cough:

- **Honey & Lemon**: Mix one tablespoon of honey with warm water and lemon juice. This soothes the throat and reduces coughing. 🍋
- **Tulsi (Holy Basil) Tea**: Boil tulsi leaves in water and drink it. This helps with respiratory issues. 🌿

--- Retrieved Documents ---
Content: and sore throat.
 Turmeric Milk: Mix 1 tsp turmeric in warm milk. Acts as an anti-inflammatory and
boosts immunity.
 Steam Inhalation: Boil water, add eucalyptus oil, and inhale steam to clear nasal
congestion.
2.2 Cough
 Honey & Lemon: Mix 1 tbsp honey with warm water and lemon juice. Soothes the
throat and reduces cough.
 Tulsi (Holy Basil) Tea: Boil tulsi leaves in water and drink. Helps with respiratory
issues.
2.3 Headache
 Peppermint Oil: Apply a few drops on the temples for relief.
Source: /content/GL/LLM/RAG/Basic_Home_Rem

In [ ]:
print("\n--- Cleaning up. Deleting the index... ---")
pc.delete_index(index_name)
print(f"Index '{index_name}' has been deleted.")


--- Cleaning up. Deleting the index... ---
